Import the necessary modules

In [1]:
import PyPDF2
import pandas as pd
import datetime

Extract file content into an object.

In [2]:
file = open('RD-Mortality-Report_2015-18-180531.pdf', 'rb')
fileReader = PyPDF2.PdfFileReader(file)

Extract the text from each page and cut away anything that isn't the table.

In [3]:
pdf_length = fileReader.getNumPages()
pages = []
for i in range(pdf_length):
    pages.append(fileReader.getPage(i).extractText())
    end = pages[i].find('\nTotal')
    pages[i] = pages[i][99:end].split('\n')

2016 was the only leap year so the other years have no data for September 29. We fix the list by putting in blank strings as the death toll.

In [4]:
pages[1][146] = ''
pages[1].append(70)
pages[1].append('')
pages[1].append('')

The data is seperated by month, but it needs to be seperated by rows as well to create a dataframe. Three dimensional list "year" will have 12 items, one for each month. Each month is a list with 29 - 32 items, one for each day of the month plus the header row. Each row is a list with 5 items, the first being the month column and the next four are the years 2015 - 2018.

In [5]:
year = []
for page in pages:
    count = 0
    row = []
    month = []
    for i in range(len(page)):
        count += 1
        row.append(page[i])
        if count == 5:
            month.append(row)
            count = 0
            row = []
    year.append(month)
    
year[0]

[['JAN', '2015', '2016', '2017', '2018'],
 ['1', '107', '89', '107', '95'],
 ['2', '101', '88', '108', '107'],
 ['3', '78', '79', '115', '92'],
 ['4', '121', '90', '81', '108'],
 ['5', '99', '80', '79', '94'],
 ['6', '104', '96', '90', '87'],
 ['7', '79', '92', '116', '97'],
 ['8', '73', '77', '108', '85'],
 ['9', '90', '86', '81', '91'],
 ['10', '75', '90', '85', '89'],
 ['11', '88', '86', '77', '94'],
 ['12', '85', '81', '75', '92'],
 ['13', '74', '79', '84', '107'],
 ['14', '98', '87', '103', '90'],
 ['15', '88', '94', '104', '93'],
 ['16', '111', '99', '103', '103'],
 ['17', '92', '92', '85', '83'],
 ['18', '90', '83', '94', '96'],
 ['19', '101', '81', '91', '75'],
 ['20', '98', '100', '88', '99'],
 ['21', '105', '81', '102', '91'],
 ['22', '71', '87', '90', '95'],
 ['23', '82', '92', '95', '67'],
 ['24', '84', '100', '87', '93'],
 ['25', '83', '92', '96', '86'],
 ['26', '83', '98', '94', '93'],
 ['27', '73', '91', '81', '86'],
 ['28', '87', '98', '99', '91'],
 ['29', '73', '92', '

List of lists is turned into a list of dataframes.

In [6]:
df_months = []
for month in year:
    df = pd.DataFrame(month[1:], columns = [month[0][0], '2015', '2016', '2017', '2018'])
    df_months.append(df)

In [7]:
df_months[0]

,JAN,2015,2016,2017,2018
0,1,107,89,107,95
1,2,101,88,108,107
2,3,78,79,115,92
3,4,121,90,81,108
4,5,99,80,79,94
5,6,104,96,90,87
6,7,79,92,116,97
7,8,73,77,108,85
8,9,90,86,81,91
9,10,75,90,85,89


Turn month into integers, add them as a column.

In [8]:
for i in range(12):
    year[i][0][0] = i + 1
    for row in year[i]:
        row.insert(0, year[i][0][0])
    year[i].pop(0)

Function to get list of all items in a column.

In [9]:
def getColumn(index):
    arr = []
    for month in year:
        for day in month:
            arr.append(day[index])
    return arr

Get each column of "year".

In [10]:
colMonth = getColumn(0)
colDay = getColumn(1)
col2015 = getColumn(2)
col2016 = getColumn(3)
col2017 = getColumn(4)
col2018 = getColumn(5)

Make new list table with years in ascending order.

In [11]:
def getYear(year, deaths):
    table = []
    for i in range(366):
        row = []
        row.append(colMonth[i])
        row.append(colDay[i])
        row.append(year)
        row.append(deaths[i])
        table.append(row)
    return table

a2015 = getYear('2015', col2015)
a2016 = getYear('2016', col2016)
a2017 = getYear('2017', col2017)
a2018 = getYear('2018', col2018)
table = a2015 + a2016 + a2017 + a2018

Add datetime column to each row.

In [12]:
for row in table:
    if row[2] != '2016' and row[0] == 2 and row[1] == '29':
        d = None
    else:
        d = datetime.date(int(row[2]), row[0], int(row[1]))
    row.append(d)

Create new dataframe.

In [13]:
df = pd.DataFrame(table, columns = ['month', 'day', 'year', 'deaths', 'date'])

In [14]:
df

,month,day,year,deaths,date
0,1,1,2015,107,2015-01-01
1,1,2,2015,101,2015-01-02
2,1,3,2015,78,2015-01-03
3,1,4,2015,121,2015-01-04
4,1,5,2015,99,2015-01-05
...,...,...,...,...,...
1459,12,27,2018,0,2018-12-27
1460,12,28,2018,0,2018-12-28
1461,12,29,2018,0,2018-12-29
1462,12,30,2018,0,2018-12-30


In [15]:
df.to_csv(r'mortality-rates.csv')